<h1> Homework 6 </h1>

Problem 1: The Resilient Data Cleaner
Concept: try-except, String parsing, List of Dictionaries. The Task: You have a file raw_data.txt that is supposed to contain student data in the format: Name, Age, Score.

However, the file is corrupted. Some lines are missing commas, some ages are written as words (e.g., "twenty"), and some scores are missing.

Your Goal:
Read the file line by line.
Use a try-except block inside the loop to catch errors when converting Age or Score to integers.

If a line is valid, add it to a list of dictionaries.
If a line is invalid, write that specific line to a separate error log file called errors.log.
Print the count of valid vs. invalid records.

Expected Output (errors.log):


Bob, twenty, 90
Charlie, 22

In [2]:
valid_data = []
error_count = 0

try:
    with open("raw_data.txt", "r") as infile, open("errors.log", "w") as errfile:
        for line in infile:
            line = line.strip()
            parts = line.split(",")
            
            try:
                if len(parts) != 3:
                    raise ValueError("Wrong number of columns")
                
                name = parts[0].strip()
                age = int(parts[1].strip())
                score = int(parts[2].strip())
                
                valid_data.append({"name": name, "age": age, "score": score})
                
            except ValueError:
                errfile.write(line + "\n")
                error_count += 1

    print(f"Processed. Valid: {len(valid_data)}, Errors: {error_count}")

except FileNotFoundError:
    print("Could not find raw_data.txt")

Processed. Valid: 9, Errors: 4


Problem 2: The "Safe" Shopping Cart (KeyError + Finally)
Concept: Dictionary lookups, KeyError, finally. The Task: You have a dictionary of item prices: prices = {'apple': 1.5, 'milk': 2.5, 'bread': 2.0}.

You are reading a user's shopping list from a file list.txt.


Your Goal:
Read items from list.txt.
Try to look up the price in the dictionary and add it to a total_cost.
If the item is not in the dictionary (KeyError), catch the error and print "Item [name] not found."
Crucially: Use a finally block to print "Checked item: [name]" after every single lookup, regardless of whether it succeeded or failed.

Console Output:
Checked item: apple
Item eggs not found.
Checked item: eggs
Checked item: milk
Total: 4.0

In [17]:
prices = {'apple': 1.5, 'milk': 2.5, 'bread': 2.0, 'eggs': 3.0, 'cheese': 5.0}
total = 0

try:
    with open("list.txt", "r") as file:
        # print(type(file))
        for line in file:
            item = line.strip()
            try:
                total += prices[item]
            except KeyError:
                print(f"Item {item} not found.")
            finally:
                print(f"Checked item: {item}")
                
    print(f"Total: {total}")
except FileNotFoundError:
    print("List file missing!")

Checked item: apple
Checked item: eggs
Checked item: milk
Item soda not found.
Checked item: soda
Checked item: bread
Item chips not found.
Checked item: chips
Checked item: cheese
Checked item: apple
Item candy not found.
Checked item: candy
Item water not found.
Checked item: water
Checked item: eggs
Item steak not found.
Checked item: steak
Total: 18.5


Problem 3: The Log File deduplicator (Sets + Files)
Concept: Sets for uniqueness, File Writing. The Task: You have a server log access.log. Every line starts with an IP address, followed by other info.

Your Goal:

Extract just the IP address (the first word) from every line.

Use a Set to store only the unique visitors.

Write these unique IPs to a new file unique_visitors.txt.

Robustness: Wrap the whole file opening logic in a try-except block to handle FileNotFoundError in case access.log doesn't exist.

Output
192.168.1.1
192.168.1.2

In [24]:
unique_ips = set()

try:
    with open("access.log", "r") as f:
        for line in f:
            parts = line.strip().split()
            # print(parts)
            if parts:
                unique_ips.add(parts[0])

    with open("unique_visitors.txt", "w") as f:
        for ip in unique_ips:
            f.write(ip + "\n")
            
    print(f"Extraction complete. Found {len(unique_ips)} unique IPs.")

except FileNotFoundError:
    print("Error: access.log not found.")

Extraction complete. Found 6 unique IPs.


Problem 4: The Merged Report (Multi-File + Tuples)
Concept: Reading multiple files, merging data, Tuple immutability. The Task: You have math_scores.txt and science_scores.txt. Both follow the format Name,Score.

Your Goal:
Read both files.
Create a dictionary where the Key is the Student Name and the Value is a Tuple: (MathScore, ScienceScore).
If a student exists in one file but not the other, record their missing score as 0.
Constraint: You must handle potential empty lines or bad integers in the files gracefully without crashing.


Expected Output (Dict):
Python
{'Alice': (90, 95), 'Bob': (80, 0), 'Charlie': (0, 70)}

In [25]:
report = {}

def process_file(filename, subject_index):
    try:
        with open(filename, "r") as f:
            for line in f:
                try:
                    name, score_str = line.strip().split(",")
                    score = int(score_str)
                    
                    if name not in report:
                        report[name] = [0, 0]
                    
                    report[name][subject_index] = score
                except ValueError:
                    continue
    except FileNotFoundError:
        print(f"File {filename} missing.")

process_file("math.txt", 0)
process_file("science.txt", 1)

final_report = {k: tuple(v) for k, v in report.items()}
import pprint
pprint.pprint(final_report)

{'Alice': (90, 95),
 'Bob': (80, 0),
 'Charlie': (0, 70),
 'David': (75, 80),
 'Eve': (88, 90),
 'Frank': (92, 85),
 'Grace': (100, 0),
 'Heidi': (0, 88),
 'Ivan': (60, 65),
 'Judy': (85, 82),
 'Kevin': (70, 75),
 'Leo': (95, 90),
 'Mallory': (0, 85)}


Problem 5: The "Strict" Config Loader
Concept: raise, Custom Logic, Parsing.

The Task: You are writing a parser for a game configuration file settings.cfg (Format: KEY=VALUE).

Your Goal:

Read the file. Split lines by =.

Raise and Catch errors for specific rules:

If a line has no =, raise a ValueError.

If the key is "MAX_PLAYERS" and the value is > 100, raise a ValueError ("Too many players").

If the key is "DIFFICULTY", allow only "Easy" or "Hard". If it's anything else, don't crash—just force it to "Medium" (silent correction).
Store valid settings in a dictionary.

In [26]:
config = {}

try:
    with open("settings.cfg", "r") as f:
        for line in f:
            line = line.strip()
            # Skip comments or empty lines
            if not line or line.startswith("#"):
                continue
                
            try:
                if "=" not in line:
                    raise ValueError("Missing '='")
                
                key, val = line.split("=", 1)
                key = key.strip()
                val = val.strip()
                
                # Logic Validations
                if key == "MAX_PLAYERS" and int(val) > 100:
                    raise ValueError(f"Too many players ({val})")
                
                if key == "DIFFICULTY":
                    if val not in ["Easy", "Hard"]:
                        val = "Medium" # Silent fix
                
                config[key] = val
                
            except ValueError as e:
                print(f"Skipping line '{line}': {e}")

    print("\nFinal Config:", config)

except FileNotFoundError:
    print("Config file missing.")

Skipping line 'MAX_PLAYERS=150': Too many players (150)
Skipping line 'invalid_line_here': Missing '='
Skipping line 'MAX_PLAYERS=200': Too many players (200)

Final Config: {'DIFFICULTY': 'Medium', 'SOUND': 'On', 'GRAPHICS': 'High', 'PORT': '8080', 'TIMEOUT': '30', 'MAX_PLAYERS': '50', 'SERVER_NAME': 'PythonCity', 'MUSIC': 'Off', 'FOV': '90'}
